In [2]:
import pandas as pd
from pandas import Series, DataFrame

In [3]:
# Covid-19 데이터 분석 시각화
# plotly를 통한 시각화
# Dash 라이브러리를 통한 웹 서비스
# 01. Covid-19 데이터 전처리 및 분석
# 02. Covid-19 데이터 시각화(feat. plotly)
# 03. plotly + Dash로 데이터 분석 웹서버 구축하기

In [4]:
# 01. Covid-19 데이터 전처리 및 분석
# Covid-19 데이터 분석 및 plotly 시각화 실습하기
# 데이터 분석
# 시각화
# 웹서버 구축 및 웹서비스

## 1. Covid-19 데이터 가져오기
- https://github.com/owid/covid-19-data/tree/master/public/data : 매일 업데이트된 파일을 제공함

In [5]:
covid = pd.read_excel('data/owid-covid-data.xlsx')

## 2. 데이터 탐색 및 전처리

##### 데이터 크기, 컬럼들의 개수와 타입 등 확인

In [6]:
# Covid-19 데이터는 약 200개 이상의 국가에서 2020년 1월부터 수집된 데이터로써, 약 60개 컬럼으로 구성되어 있음
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137839 entries, 0 to 137838
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    137839 non-null  object 
 1   continent                                   129066 non-null  object 
 2   location                                    137839 non-null  object 
 3   date                                        137839 non-null  object 
 4   total_cases                                 130171 non-null  float64
 5   new_cases                                   130166 non-null  float64
 6   new_cases_smoothed                          129126 non-null  float64
 7   total_deaths                                118870 non-null  float64
 8   new_deaths                                  119066 non-null  float64
 9   new_deaths_smoothed                         129126 non-null  float64
 

##### date 컬럼을 datetime으로 변경하기

In [7]:
# to_datetime() 함수
# 컬럼의 데이터타입을 datetime으로 변환
covid['date'] =  pd.to_datetime(covid['date'], format='%Y-%m-%d')

In [8]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137839 entries, 0 to 137838
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   iso_code                                    137839 non-null  object        
 1   continent                                   129066 non-null  object        
 2   location                                    137839 non-null  object        
 3   date                                        137839 non-null  datetime64[ns]
 4   total_cases                                 130171 non-null  float64       
 5   new_cases                                   130166 non-null  float64       
 6   new_cases_smoothed                          129126 non-null  float64       
 7   total_deaths                                118870 non-null  float64       
 8   new_deaths                                  119066 non-null  float64      

##### 년도(year), 달(month), 일(day), 주차(WeekNumber, %U)과 요일(weekDay, %a) 컬럼 추가하기

In [9]:
covid['year'] = covid['date'].apply(lambda x: x.strftime('%Y'))
covid['month'] = covid['date'].apply(lambda x: x.strftime('%m'))
covid['day'] = covid['date'].apply(lambda x: x.strftime('%d'))
covid['weekNumber'] = covid['date'].apply(lambda x: x.strftime('%U'))
covid['weekDay'] = covid['date'].apply(lambda x: x.strftime('%a'))

In [10]:
covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,year,month,day,weekNumber,weekDay
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,0.511,NaN,NaN,NaN,NaN,2020,02,24,08,Mon
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,0.511,NaN,NaN,NaN,NaN,2020,02,25,08,Tue
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,0.511,NaN,NaN,NaN,NaN,2020,02,26,08,Wed
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,0.511,NaN,NaN,NaN,NaN,2020,02,27,08,Thu
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,0.511,NaN,NaN,NaN,NaN,2020,02,28,08,Fri


##### date를 row index로 설정하기

In [11]:
covid2 = covid.set_index(['date']).sort_index()

In [12]:
# DatetimeIndex인지 확인
type(covid2.index)

pandas.core.indexes.datetimes.DatetimeIndex

##### 데이터 분석에 활용할 데이터의 범위 정하기 (나라, 기간)

1) 수집되고 있는 나라의 개수 확인

In [13]:
# 결과 목록의 개수(size)가 수집 국가의 개수
covid2.location.unique().size

237

In [14]:
# World, Asia, European Union 등 나라가 아닌 대륙 정보가 혼재되어 있음
# Location 컬럼에는 나라와 대륙 정보가 혼재되어 있으므로, 대륙 정보를 삭제해야 함
covid2.location.Unique()

AttributeError: 'Series' object has no attribute 'Unique'

In [ ]:
# continent 컬럼의 값 = null
# location 컬럼의 값이 대륙임을 알 수 있음
covid2[covid2.continent.isnull()].location.unique()

In [ ]:
covid2[covid2.continent.isnull()]

In [ ]:
covid2[covid2.location == 'Asia']

In [ ]:
# 더 정확히 알아보기
# continent(대륙) 컬럼이 null이 아닌 데이터만 선택하면 정확한 나라의 개수를 알 수 있음
# 224개의 나라에서 데이터 수집 중
covid2[covid2.continent.notnull()].location.unique().size

2) 데이터 수집 기간 확인

In [ ]:
print(covid2.index.min(), covid2.index.max())

3) 날짜별 데이터 수집 개수 확인

In [ ]:
# 정보를 전체 확인하고 싶다면?
# pandas의 set_option에서 display.max_rows를 늘리면 됨
# 출력되는 컬럼의 개수를 늘리고자 한다면, display.max_columns의 값을 수정하면 됨

# 결과에 표현되는 최대 row 갯수
pd.set_option('display.max_rows', 500)

# 결과에 표현되는 최대 columns 갯수
#pd.set_option('display.max_columns', 100)

In [ ]:
covid2.index.value_counts().sort_index()

In [ ]:
# 데이터 분석에 활용할 데이터의 범위 정하기(나라, 기간)

4) 코로나 바이러스가 가장 많이 걸린 상위 100개 나라만 선택

In [ ]:
covid2[covid2.location == 'South Korea'] # total_cases가 누적 값인 것을 확인

In [ ]:
# 1. covid2.index.max() 가장 최근 날짜 구하기
covid2.index.max()

In [ ]:
# 2. 가장 최신 날짜의 데이터만 선택
# recent : 최신 데이터 저장
recent = covid2[covid2.index == covid2.index.max()]
recent

In [ ]:
# 3. 값이 가장 큰 100개의 나라(location) 선택
top100 = recent.nlargest(100, 'total_cases').location.values
top100

In [ ]:
# 4. covid2에서 top100 나라 데이터만 선택하여, covid2에 다시 저장하기
covid2 = covid2[covid2.location.isin(top100)]

5) 분석에 활용할 데이터의 수집 기간 정하기

In [ ]:
# 2020-03-27~2021-12-04까지는 빠짐 없이 수집됨을 확인하였으므로, 해당 기간의 데이터만 활용하기로 함
covid2.index.value_counts().sort_index()

In [ ]:
data = covid2['2020-03-27':'2021-12-04']

In [ ]:
data # 최종 데이터

## 3. 데이터 분석

#### [실습 #1] 누적 확진자수가 가장 많은 국가 10개 찾아보기

In [ ]:
# 조건 색인
# 01. 제일 최근 데이터
# data.index == data.index.max()
# 02. 대륙 데이터 제외
# data.continent.notnull()

In [ ]:
data[(data.index == data.index.max()) & (data.continent.notnull())].nlargest(10, 'total_cases')

#### [실습 #2] 2021-11 한 달 동안(21-11-01~21-11-31) 가장 많은 확진자가 발생한 국가 10개 찾아보기

In [ ]:
data

In [ ]:
# new_cases 값을 활용하여 한 달 동안의 신규 확진자 수를 구할 수 있음
# 조건 색인
# 01. 대륙 데이터 제외
# 02. 11월 데이터 선택
# 데이터 집계하기
# 확진자 발생 상위 10개 국가 구하기
ex2 = data[data.continent.notnull()]['2021-11']\
    .pivot_table(index = 'location', values = ['new_cases', 'new_deaths'], aggfunc = 'sum')\
    .nlargest(10, 'new_cases')

In [ ]:
ex2

#### [실습 3] 실습 2에서 구한 나라들의 2021년 1월부터 2021년 11월까지의 월별 신규확진자수, 사망자수, 검사자수 구하기

In [ ]:
# 1) data에서 실습2에서 구한 나라들의 데이터만 선택
ex3 = data[data.location.isin(ex2.index)]

In [ ]:
# 2) 2021년 1월부터 2021년 11월 데이터만 선택
# ex3에는 2021년 1월부터 11월까지 최근 신규 확진자 수가 많이 발생한 상위 10개 국가의 데이터가 저장되어 있음
ex3 = ex3['2021-01':'2021-11']

In [ ]:
# 3) 월별 신규확진자수, 사망자수, 검사자수 구하기
ex3 = ex3.pivot_table(index = 'month', columns = 'location', aggfunc = 'sum', values = ['new_cases', 'new_deaths', 'new_tests'])

In [ ]:
ex3

In [ ]:
# 02. Covid-19 데이터 시각화(feat. plotly)
# Matplotlib으로 그리기
pd.options.plotting.backend = 'matplotlib'

In [ ]:
ex3['new_cases'].plot()

In [ ]:
# 4) plotly로 그리기
# Error 발생
# plotly.express는 아직 multiIndex(계층 색인)은 지원하지 않음
pd.options.plotting.backend = "plotly"
fig1 = ex3.plot()
fig1.show()

In [ ]:
# 4) plotly로 그리기
# 우회 방법
# 계층 색인을 없애고 최상위 색인으로 하나씩 선택하기
# 3개의 case에 대해 각각 figure을 그릴 수 있음
# new_cases
pd.options.plotting.backend = "plotly"
fig1 = ex3['new_cases'].plot(width = 700, height = 400)
fig1.show()

In [ ]:
# new_deaths 
fig2 = ex3['new_deaths'].plot()
fig2.show()

In [ ]:
# new_tests
fig3 = ex3['new_tests'].plot()
fig3.show()

In [ ]:
ex3

In [ ]:
ex3.to_excel('data/covid_top10.xlsx')

In [ ]:
# 이 부분은 하지 말자..## 

In [ ]:
ex3_1 = ex3.stack(0).reset_index()

In [ ]:
ex3_1.columns[2:12]

In [ ]:
# 하나의 figure 안에 각각의 subplot으로 그리기
ex3_1.plot(x='month',y = ex_3_1.columns[2:12],
                               facet_row = 'level_1')

In [ ]:
# 5) 그래프를 html로 export 하기
# 시각화된 그래프를 html로 export가 쉽게 가능
# write_html 함수를 실행하여 html로 저장하기
# 웹 서버에서 분석결과가 저장된 html 문서들의 링크를 사용하면, 분석 결과를 웹에서 볼 수 있음
fig1.write_html('covid_top10_countries_new_cases.html')
fig2.write_html('covid_top10_countries_new_deaths.html')
fig3.write_html('covid_top10_countries_new_tests.html')

In [ ]:
# 03. plotly + Dash로 데이터 분석 웹서버 구축하기
# Dash를 통한 웹 서비스 구축하기
# Sample 코드 실행 -> 웹 서버가 내 PC에서 구동 -> 웹 서버를 통한 웹 서비스 가능
# 화면 구성 : layout
# 화면에 그려질 그래프 : callback

In [ ]:
# Dash
# 대시보드 형태로 데이터 시각화된 결과를 웹 서버로 쉽게 구축할 수 있는 외부 라이브러리
# 대시보드는 다양한 형태의 그래프를 한 웹 페이지 내에서 보여주는 것이 가장 기본적인 형태

## 4. 데이터 분석 결과로 웹 서버 구축하기 with Dash
 - https://plotly.com/python/getting-started/

In [ ]:
# https://plotly.com/python/getting-started/
# 웹 브라우저에 접속하여 분석 결과 데이터가 그래프로 그려지는 지 실습하기
# Dash 코드를 실행하면 웹 서비스를 실행할 수 있음
# 관리자 권한으로 Anaconda Prompt를 실행해서 pip install dash하면 설치가 완료됨
# flask : 웹 서버 프레임워크
# Dash를 사용하면 flask와 같은 별도 프레임워크 없이, 웹 서버를 간단하게 구축할 수 있음
# Error 발생
# ctrl + c해서 파이썬 파일로 만들면 됨
# 다운로드 후 파이썬 파일로 저장한 다음 파이썬 파일을 실행
# python <파일명>을 실행하면 웹 서버가 구동
# localhost:8050으로 접속하면 sample 화면을 확인할 수 있음
# 웹 서버를 통해 다른 장소나 디바이스에서도 화면을 똑같이 볼 수 있고, 다른 사람과의 공유가 용이

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("Color:"),
    dcc.Dropdown(
        id="dropdown",
        options=[
            {'label': x, 'value': x}
            for x in ['Gold', 'MediumTurquoise', 'LightGreen']
        ],
        value='Gold',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("dropdown", "value")])
def display_color(color):
    fig = go.Figure(
        data=go.Bar(y=[2, 3, 1], marker_color=color))
    return fig

app.run_server(debug=True)

In [ ]:
covid19_top10 = pd.read_excel('data/covid_top10.xlsx', header = [0,1])

In [ ]:
covid19_top10

In [ ]:
covid19_top10.set_index(covid19_top10.columns[0])[1:]
# 데이터

In [ ]:
# https://plotly.com/python/getting-started/
# 웹 브라우저에 접속하여 분석 결과 데이터가 그래프로 그려지는 지 실습하기
# Dash 코드를 실행하면 웹 서비스를 실행할 수 있음
# 관리자 권한으로 Anaconda Prompt를 실행해서 pip install dash하면 설치가 완료됨
# flask : 웹 서버 프레임워크
# Dash를 사용하면 flask와 같은 별도 프레임워크 없이, 웹 서버를 간단하게 구축할 수 있음
# Error 발생
# ctrl + c해서 파이썬 파일로 만들면 됨
# 다운로드 후 파이썬 파일로 저장한 다음 파이썬 파일을 실행
# python <파일명>을 실행하면 웹 서버가 구동
# localhost:8050으로 접속하면 sample 화면을 확인할 수 있음
# 웹 서버를 통해 다른 장소나 디바이스에서도 화면을 똑같이 볼 수 있고, 다른 사람과의 공유가 용이

# 저장한 파일을 읽어 그래프를 그리고 웹 서비스하기
# 3개의 html 파일을 한 번에 볼 수 있는 대시보드 만들기
# 01. Pandas의 plotly 기능 사용을 위한 내용 추가 및 파일 읽어 오기, 데이터 가공 구문 추가
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
pd.options.plotting.backend = 'plotly'

app = dash.Dash(__name__)
# 웹 서버가 구동될 때, 저장된 분석 결과를 읽어오는 과정
# 저장된 분석결과 데이터를 읽어서 data에 저장
data = pd.read_excel('data/covid_top10.xlsx', header = [0, 1])
# 첫 번째 컬럼을 로우 인덱스로 변경
data = data.set_index(data.columns[0])[1:]

# 02. html 관련 코드에 원하는 내용으로 변경하기
app.layout = html.Div([
    html.P("Type:"), # 01. 데이터 타입 변경
    dcc.Dropdown(
        id="dropdown",
        options=[
            {'label': x, 'value': x}
            for x in ['new_cases', 'new_deaths', 'new_tests'] # 02. 3개의 컬럼명 변경
        ],
        value='new_cases',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])

# 03. 분석 결과를 그래프로 보여주기
@app.callback(
    Output("graph", "figure"), 
    [Input("dropdown", "value")])
# 함수 변경하기
def display_color(val): # 선택한 값(val)에 해당하는 데이터의 그래프를 그림
    fig = data[val].plot()
    return fig

app.run_server(debug=True)